<a href="https://colab.research.google.com/github/Leonardoup11/SeniorProject/blob/main/Senior_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install** **tflearn**

In [9]:
!pip install tflearn

**Install chatterbot**

In [10]:
!pip install chatterbot

**Import packages and libraries**

In [11]:
from flask import Flask, render_template, request
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
from chatterbot.trainers import ListTrainer

app = Flask(__name__)


import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import tflearn
import numpy
import json
import pandas as pd
import pickle
import tensorflow
import random

#download punk

nltk.download('punkt')


#Import URL from GitHub JSON file
url = ("https://raw.githubusercontent.com/Leonardoup11/peddrinnn/main/lenguage.json")
intents = pd.read_json(url)

#Name JSON FILE
data = intents
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1
# Train Data
        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

from tensorflow.python.framework import ops
ops.reset_default_graph()


# Activation Function SOFTMAX

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Training the model**

In [12]:


model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")




Training Step: 13999  | total loss: 0.03218 | time: 0.094s
| Adam | epoch: 1000 | loss: 0.03218 - acc: 0.9830 -- iter: 104/109
Training Step: 14000  | total loss: 0.02980 | time: 0.104s
| Adam | epoch: 1000 | loss: 0.02980 - acc: 0.9847 -- iter: 109/109
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [13]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

**Input recognization / Answers**

In [ ]:
 def chat():
    print("Start talking with Pedro your AI assistant (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))
chat()

Start talking with Pedro your AI assistant (type quit to stop)!
 I hope all is going well, how may I help you
Hello! how can I assist you?
I'm Pedro!
My name is Pedro.
It's been a rough day but so far so good, how about yourself?
Smoothies and Cookies!
The smoothie is $5 dollars and the cookie is $2 dolars
Hi there, how can I help?
I'm Pedro!
How could I help you today
I am doing great! How are you? 
Smoothies and Cookies!
The smoothie is $5 dollars and the cookie is $2 dolars
Urbiola's is open Monday through Saturday from 4pm to 12am 
The weather is nice over here, perfect for a fresh smoothie and a delicious cookie
How do billboards talk? Sign language
What do you call an ant who fights crime? A vigilANTe!
Did you know that The entire world's population could fit inside Los Angeles
It's been a rough day but so far so good, how about yourself?
The smoothie is $5 dollars and the cookie is $2 dolars
We do have great promotions! specially on mondays and tuesdays, buy one cookie get one f